In [2]:
# Import Libraries
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

# Define updated classes and functions
class Time_Series_Dataset(Dataset):
    def __init__(self, inputs, decoder_inputs, outputs):
        self.inputs = inputs
        self.decoder_inputs = decoder_inputs
        self.outputs = outputs

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        x = self.inputs[idx]
        decoder_input = self.decoder_inputs[idx]
        y = self.outputs[idx]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(decoder_input, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=2):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

    def forward(self, x):
        batch_size = x.size(0)
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(x.device)
        cell = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(x.device)
        out, (hidden, cell) = self.lstm(x, (hidden, cell))
        return hidden, cell

class Decoder(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_quantiles, num_layers=2):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size * num_quantiles)
        self.num_quantiles = num_quantiles

    def forward(self, x, hidden, cell):
        out, (hidden, cell) = self.lstm(x, (hidden, cell))
        out = self.fc(out)
        out = out.view(out.size(0), out.size(1), self.num_quantiles)
        return out, hidden, cell

class EncoderDecoderLSTM(nn.Module):
    def __init__(self, input_size, output_size, hidden_size, num_layers, num_quantiles):
        super(EncoderDecoderLSTM, self).__init__()
        self.encoder = Encoder(input_size, hidden_size, num_layers)
        self.decoder = Decoder(output_size, hidden_size, output_size, num_quantiles, num_layers)

    def forward(self, encoder_inputs, decoder_inputs):
        hidden, cell = self.encoder(encoder_inputs)
        decoder_inputs = decoder_inputs.unsqueeze(-1)  # Add feature dimension to decoder inputs
        outputs, hidden, cell = self.decoder(decoder_inputs, hidden, cell)
        return outputs

def quantile_loss(preds, targets, quantiles):
    losses = []
    for i, quantile in enumerate(quantiles):
        errors = targets[:, :, i] - preds[:, :, i]
        losses.append(torch.mean(torch.max((quantile - 1) * errors, quantile * errors)))
    return torch.mean(torch.stack(losses))

def split_data(features, target, input_steps, output_steps, train_ratio, seed):
    X, y, decoder_inputs = [], [], []
    total_size = input_steps + output_steps
    for i in range(len(features) - total_size + 1):
        X.append(features[i:i + input_steps])
        y.append(target[i + input_steps:i + total_size])
        decoder_inputs.append(target[i + input_steps - 1:i + input_steps + output_steps - 1])
    
    X_train, X_test, y_train, y_test, decoder_inputs_train, decoder_inputs_test = train_test_split(
        X, y, decoder_inputs, train_size=train_ratio, random_state=seed
    )
    return X_train, X_test, y_train, y_test, decoder_inputs_train, decoder_inputs_test

Bitcoin = pd.read_csv('data/coin_Bitcoin.csv')
data = Bitcoin.iloc[:, 4:]
features = data[['Close']]  # Multivariate input
features = MinMaxScaler().fit_transform(features)  # Normalize input
target = data['Close']
target_reshaped = np.array(target).reshape(-1, 1)  # Normalize output
scaler = MinMaxScaler(feature_range=(0, 1))
target = scaler.fit_transform(target_reshaped).flatten()

input_steps = 6
output_steps = 5
train_ratio = 0.8
seed = 5925
quantiles = [0.05, 0.25, 0.5, 0.75, 0.95]
num_experiments = 30  # Default: 30 (change to lower value for less intensive runs)

rmse, mae, mape = [], [], []
rmse_005, rmse_025, rmse_050, rmse_075, rmse_095 = [], [], [], [], []
rmse_steps = [[] for _ in range(output_steps)]
mae_steps = [[] for _ in range(output_steps)]
mape_steps = [[] for _ in range(output_steps)]

train_rmse, train_mae, train_mape = [], [], []
train_rmse_steps = [[] for _ in range(output_steps)]
train_mae_steps = [[] for _ in range(output_steps)]
train_mape_steps = [[] for _ in range(output_steps)]

for exp in range(num_experiments):
    X_train, X_test, y_train, y_test, decoder_inputs_train, decoder_inputs_test = split_data(
        features, target, input_steps, output_steps, train_ratio, seed
    )
    
    train_dataset = Time_Series_Dataset(X_train, decoder_inputs_train, y_train)
    test_dataset = Time_Series_Dataset(X_test, decoder_inputs_test, y_test)
    
    train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=False)
    test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)
    
    # Define our parameters
    input_size = features.shape[1]  # Multivariate input
    output_size = 1  # Predicting one value per time step
    hidden_size = 100
    num_layers = 2
    num_quantiles = len(quantiles)
    
    model = EncoderDecoderLSTM(input_size, output_size, hidden_size, num_layers, num_quantiles)
    
    # Loss and optimizer
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    
    # Training loop
    num_epochs = 100  # Default: 100
    for epoch in range(1, num_epochs + 1):
        model.train()
        for encoder_inputs, decoder_inputs, targets in train_dataloader:
            # Determine batch size dynamically
            batch_size = encoder_inputs.shape[0]
    
            # Correctly reshape the inputs
            encoder_inputs = encoder_inputs.view(batch_size, input_steps, input_size)
            decoder_inputs = decoder_inputs.view(batch_size, output_steps)
            targets = targets.view(batch_size, output_steps, 1)  # Add feature dimension
    
            # Forward pass
            outputs = model(encoder_inputs, decoder_inputs)
    
            # Reshape outputs to match targets
            outputs = outputs.view(batch_size, output_steps, num_quantiles)
            targets = targets.repeat(1, 1, num_quantiles)  # Expand targets to match the number of quantiles
    
            # Compute loss
            loss = quantile_loss(outputs, targets, quantiles)
    
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    
    # Evaluate the model on the test set
    model.eval()
    y_pred = []
    y_test = []
    
    with torch.no_grad():
        for encoder_inputs, decoder_inputs, targets in test_dataloader:
            # Determine batch size dynamically
            batch_size = encoder_inputs.shape[0]
    
            # Correctly reshape the inputs
            encoder_inputs = encoder_inputs.view(batch_size, input_steps, input_size)
            decoder_inputs = decoder_inputs.view(batch_size, output_steps)
            targets = targets.view(batch_size, output_steps, 1)  # Add feature dimension
    
            # Forward pass
            outputs = model(encoder_inputs, decoder_inputs)
            outputs = outputs.view(batch_size, output_steps, num_quantiles)  # Reshape outputs
    
            y_pred.append(outputs.numpy())
            y_test.append(targets.repeat(1, 1, num_quantiles).numpy())  # Expand targets to match the number of quantiles
    
    # Convert lists to numpy arrays
    predicted_values = np.concatenate(y_pred, axis=0)
    actuals = np.concatenate(y_test, axis=0)[:, :, 0]

    pred_005 = predicted_values[:, :, 0]
    pred_025 = predicted_values[:, :, 1]
    pred_050 = predicted_values[:, :, 2]
    pred_075 = predicted_values[:, :, 3]
    pred_095 = predicted_values[:, :, 4]
    
    # Calculate RMSE here
    mse_005 = mean_squared_error(pred_005, actuals)
    mse_025 = mean_squared_error(pred_025, actuals)
    mse_050 = mean_squared_error(pred_050, actuals)
    mse_075 = mean_squared_error(pred_075, actuals)
    mse_095 = mean_squared_error(pred_095, actuals)

    rmse_005.append(np.sqrt(mse_005))
    rmse_025.append(np.sqrt(mse_025))
    rmse_050.append(np.sqrt(mse_050))
    rmse_075.append(np.sqrt(mse_075))
    rmse_095.append(np.sqrt(mse_095))

    pred_values = predicted_values.reshape(-1, 1)
    pred_values = scaler.inverse_transform(pred_values)
    pred_values = pred_values.reshape(predicted_values.shape)
    actual_values = scaler.inverse_transform(actuals)
    
    predicted_005 = pred_values[:, :, 0]
    predicted_025 = pred_values[:, :, 1]
    predicted_050 = pred_values[:, :, 2]
    predicted_075 = pred_values[:, :, 3]
    predicted_095 = pred_values[:, :, 4]

    mae.append(mean_absolute_error(actual_values, predicted_050))
    mape.append(mean_absolute_percentage_error(actual_values, predicted_050))

    for step in range(output_steps):
        mse_step = mean_squared_error(pred_050[:, step], actuals[:, step])
        rmse_steps[step].append(np.sqrt(mse_step))
        mae_steps[step].append(mean_absolute_error(predicted_050[:, step], actual_values[:, step]))
        mape_steps[step].append(mean_absolute_percentage_error(predicted_050[:, step], actual_values[:, step]))
        
    # Evaluate the model on the train set
    y_train_pred = []
    y_train_actual = []

    with torch.no_grad():
        for encoder_inputs, decoder_inputs, targets in train_dataloader:
            # Determine batch size dynamically
            batch_size = encoder_inputs.shape[0]
            
            # Correctly reshape the inputs
            encoder_inputs = encoder_inputs.view(batch_size, input_steps, input_size)
            decoder_inputs = decoder_inputs.view(batch_size, output_steps)  # Output steps dimension only
            targets = targets.view(batch_size, output_steps, 1)  # Add feature dimension
            
            # Forward pass
            outputs = model(encoder_inputs, decoder_inputs)
            
            # Reshape outputs to match targets
            outputs = outputs.view(batch_size, output_steps, num_quantiles)
            
            y_train_pred.append(outputs.numpy())
            y_train_actual.append(targets.repeat(1, 1, num_quantiles).numpy())  # Expand targets to match the number of quantiles

    # Convert lists to numpy arrays and remove last dimension
    y_train_pred = np.concatenate(y_train_pred, axis=0)
    y_train_actual = np.concatenate(y_train_actual, axis=0)

    # Calculate train metrics
    actuals_train = y_train_actual[:, :, 0]
    pred_train_050 = y_train_pred[:, :, 2]  # 0.50 quantile as the prediction

    train_mse = mean_squared_error(actuals_train, pred_train_050)
    train_rmse.append(np.sqrt(train_mse))

    for step in range(output_steps):
        train_mse_step = mean_squared_error(pred_train_050[:, step], actuals_train[:, step])
        train_rmse_steps[step].append(np.sqrt(train_mse_step))

    # Inverse Transform
    train_pred_values = y_train_pred.reshape(-1, 1)
    train_pred_values = scaler.inverse_transform(train_pred_values)
    train_pred_values = train_pred_values.reshape(y_train_pred.shape)
    train_actual_values = scaler.inverse_transform(actuals_train.reshape(-1, 1)).reshape(actuals_train.shape)
    
    train_predicted_050 = train_pred_values[:, :, 2]

    train_mae.append(mean_absolute_error(train_actual_values, train_predicted_050))
    train_mape.append(mean_absolute_percentage_error(train_actual_values, train_predicted_050))

    for step in range(output_steps):
        train_mae_steps[step].append(mean_absolute_error(train_actual_values[:, step], train_predicted_050[:, step]))
        train_mape_steps[step].append(mean_absolute_percentage_error(train_actual_values[:, step], train_predicted_050[:, step]))

    print(f"Experiment {exp+1}/{num_experiments} done")
    seed += 1
pm = "\00B1"

print(f"Bitcoin ED-LSTM Regression: After {num_experiments} experimental runs, here are the results:")

# Test dataset results
print(f"Across {output_steps} predictive time steps on the test dataset, " +
      f"Avg RMSE: {np.mean(rmse):.4f} {pm} {np.std(rmse):.4f}, " +
      f"Avg MAE: {np.mean(mae):.2f} {pm} {np.std(mae):.2f}, " +
      f"Avg MAPE: {np.mean(mape)*100:.3f}% {pm} {np.std(mape)*100:.3f}%")
for step in range(output_steps):
    print(
        f"At time step {step + 1} on the test dataset, "
        f"Avg RMSE: {np.mean(rmse_steps[step]):.4f} {pm} {np.std(rmse_steps[step]):.4f}, "
        f"Avg MAE: {np.mean(mae_steps[step]):.2f} {pm} {np.std(mae_steps[step]):.2f}, "
        f"Avg MAPE: {np.mean(mape_steps[step]) * 100:.3f}% {pm} {np.std(mape_steps[step]) * 100:.3f}%"
    )

# Train dataset results
print(f"Across {output_steps} predictive time steps on the train dataset, " +
      f"Avg RMSE: {np.mean(train_rmse):.4f} {pm} {np.std(train_rmse):.4f}, " +
      f"Avg MAE: {np.mean(train_mae):.2f} {pm} {np.std(train_mae):.2f}, " +
      f"Avg MAPE: {np.mean(train_mape)*100:.3f}% {pm} {np.std(train_mape)*100:.3f}%")
for step in range(output_steps):
    print(
        f"At time step {step + 1} on the train dataset, "
        f"Avg RMSE: {np.mean(train_rmse_steps[step]):.4f} {pm} {np.std(train_rmse_steps[step]):.4f}, "
        f"Avg MAE: {np.mean(train_mae_steps[step]):.2f} {pm} {np.std(train_mae_steps[step]):.2f}, "
        f"Avg MAPE: {np.mean(train_mape_steps[step]) * 100:.3f}% {pm} {np.std(train_mape_steps[step]) * 100:.3f}%"
    )


Experiment 1/30 done
Experiment 2/30 done
Experiment 3/30 done
Experiment 4/30 done
Experiment 5/30 done
Experiment 6/30 done
Experiment 7/30 done
Experiment 8/30 done
Experiment 9/30 done
Experiment 10/30 done
Experiment 11/30 done
Experiment 12/30 done
Experiment 13/30 done
Experiment 14/30 done
Experiment 15/30 done
Experiment 16/30 done
Experiment 17/30 done
Experiment 18/30 done
Experiment 19/30 done
Experiment 20/30 done
Experiment 21/30 done
Experiment 22/30 done
Experiment 23/30 done
Experiment 24/30 done
Experiment 25/30 done
Experiment 26/30 done
Experiment 27/30 done
Experiment 28/30 done
Experiment 29/30 done
Experiment 30/30 done
Bitcoin ED-LSTM Regression: After 30 experimental runs, here are the results:
Across 5 predictive time steps on the test dataset, Avg RMSE: nan  B1 nan, Avg MAE: 282.36  B1 21.39, Avg MAPE: 9.878%  B1 2.640%
At time step 1 on the test dataset, Avg RMSE: 0.0104  B1 0.0013, Avg MAE: 280.59  B1 33.79, Avg MAPE: 26.467%  B1 53.638%
At time step 2 on t

/Users/smruthirangarajan/Library/Python/3.9/lib/python/site-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/smruthirangarajan/Library/Python/3.9/lib/python/site-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/smruthirangarajan/Library/Python/3.9/lib/python/site-packages/numpy/_core/_methods.py:218: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/smruthirangarajan/Library/Python/3.9/lib/python/site-packages/numpy/_core/_methods.py:175: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/Users/smruthirangarajan/Library/Python/3.9/lib/python/site-packages/numpy/_core/_methods.py:210: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [3]:
quantile_dfs = []
for i in range(pred_values.shape[2]):
    quantile_df = pd.DataFrame(
        pred_values[:, :, i], 
        columns=[f'Quantile_{i+1}_Timestep_{j+1}' for j in range(pred_values.shape[1])]
    )
    quantile_dfs.append(quantile_df)
predicted_df = pd.concat(quantile_dfs, axis=1)

predicted_df.to_csv('bitcoin_uni_edlstm_quantile_results.csv')
pd.DataFrame(actual_values).to_csv('bitcoin_uni_edlstm_quantile_actual.csv')
pd.DataFrame(rmse_steps).transpose().to_csv('bitcoin_uni_edlstm_quantile_rmse.csv')
pd.DataFrame(mae_steps).transpose().to_csv('bitcoin_uni_edlstm_quantile_mae.csv')
pd.DataFrame(mape_steps).transpose().to_csv('bitcoin_uni_edlstm_quantile_mape.csv')

In [3]:
pm = "\u00B1"
print(f'Bitcoin ED-LSTM Average RMSE across {output_steps} time steps at different quantiles:')
print(f'Quantile 0.05: {np.mean(rmse_005):.6f} {pm} {np.std(rmse_005):.6f}')
print(f'Quantile 0.25: {np.mean(rmse_025):.6f} {pm} {np.std(rmse_025):.6f}')
print(f'Quantile 0.50: {np.mean(rmse_050):.6f} {pm} {np.std(rmse_050):.6f}')
print(f'Quantile 0.75: {np.mean(rmse_075):.6f} {pm} {np.std(rmse_075):.6f}')
print(f'Quantile 0.95: {np.mean(rmse_095):.6f} {pm} {np.std(rmse_095):.6f}')

print("--------------------------------------------")

print(f"Quantile Regression Average Performance: RMSE: {np.mean(rmse_050):.4f} {pm} {np.std(rmse_050):.4f}, MAE: {np.mean(mae):.4f} {pm} {np.std(mae):.4f}, MAPE: {np.mean(mape)*100:.4f}% {pm} {np.std(mape)*100:.4f}%")
for step in range(output_steps):
    print(f"At time step {step + 1}, predictions have RMSE: {np.mean(rmse_steps[step]):.4f} {pm} {np.std(rmse_steps[step]):.4f}, MAE: {np.mean(mae_steps[step]):.4f} {pm} {np.std(mae_steps[step]):.4f} and MAPE: {np.mean(mape_steps[step])*100:.4f}% {pm} {np.std(mape_steps[step])*100:.4f}%")


Bitcoin ED-LSTM Average RMSE across 5 time steps at different quantiles:
Quantile 0.05: 0.021962 ± 0.002737
Quantile 0.25: 0.012736 ± 0.001712
Quantile 0.50: 0.010723 ± 0.000777
Quantile 0.75: 0.012688 ± 0.001406
Quantile 0.95: 0.019992 ± 0.001838
--------------------------------------------
Quantile Regression Average Performance: RMSE: 0.0107 ± 0.0008, MAE: 283.9266 ± 20.9643, MAPE: 10.3761% ± 5.1443%
At time step 1, predictions have RMSE: 0.0103 ± 0.0012, MAE: 278.2371 ± 30.0186 and MAPE: 24.7072% ± 58.7550%
At time step 2, predictions have RMSE: 0.0105 ± 0.0011, MAE: 278.6652 ± 25.8892 and MAPE: 11.6399% ± 12.1926%
At time step 3, predictions have RMSE: 0.0109 ± 0.0012, MAE: 284.7863 ± 23.3423 and MAPE: 14.9186% ± 22.5686%
At time step 4, predictions have RMSE: 0.0110 ± 0.0012, MAE: 289.7664 ± 27.9214 and MAPE: 27.4232% ± 64.2640%
At time step 5, predictions have RMSE: 0.0107 ± 0.0014, MAE: 288.1778 ± 30.5020 and MAPE: 20.4603% ± 25.6891%
